# LSTM Architecture for Stock Market Forecasting

### Package Imports

In [2]:
from sklearn.preprocessing import MinMaxScaler
import torch 
import torch.nn as nn
import yfinance

### Load Data

In [3]:
name = 'GOOG'
ticker = yfinance.Ticker(name)
df = ticker.history(interval="1d",start="2010-04-30",end="2020-04-30")

In [6]:
# Normalize the features

scaler = MinMaxScaler(feature_range=(-1, 1))
df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [9]:
feature_columns = ['Open', 'High', 'Low', 'Close', "Volume"]
df_data = df[feature_columns]



,Open,High,Low,Close,Volume
Date,,,,,
2010-04-30,264.573303,267.836090,261.738953,-0.931808,4889057
2010-05-03,262.266968,265.464966,261.559601,-0.928080,3729527
2010-05-04,262.276917,262.386505,251.163574,-0.946515,12198151
2010-05-05,249.554611,256.897095,249.300568,-0.943935,9198750
2010-05-06,253.425095,257.793732,229.141129,-0.952373,10037683
...,...,...,...,...,...
2020-04-23,1271.550049,1293.310059,1265.670044,0.617586,1566200
2020-04-24,1261.170044,1280.400024,1249.449951,0.622168,1639600
2020-04-27,1296.000000,1296.150024,1269.000000,0.616929,1600600


In [ ]:
class LSTM_predictor(nn.Module): 
    
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM_predictor, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x): 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out